# Задание 2.2

В таблице приведены данные о содержании иммуноглобина $Ig A$ в сыворотке крови у больных пяти возрастных групп.

### Подключение библиотек и инициализация констант для реализации

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import math

LOW_INDEX = ['₀', '₁', '₂', '₃', '₄', '₅', '₆', '₇', '₈', '₉']

### Линейная регрессия

In [3]:
class LinearRegression:
    def __init__(self, Psi, Y):
        self.Psi = Psi
        self.Y = Y
    
    def beta_est(self):
        return np.linalg.inv(self.Psi.T@self.Psi)@self.Psi.T@self.Y
    
    def RSS(self):
        beta_est = self.beta_est()
        return (self.Y - self.Psi@beta_est).T@(self.Y - self.Psi@beta_est)
    
    def TSS(self):
        return np.sum([ (self.Y[i] - np.mean(self.Y))**2 for i in range(len(self.Y)) ])
    
    def e(self):
        return self.Y - self.Psi @ self.beta_est()
    
    def R2(self):
        RSS = self.RSS()
        TSS = self.TSS()
        return (TSS - RSS) / TSS
    
    def F(self):
        return self.Psi.T@self.Psi

### Плотности нормального распределения и распределений Фишера и Стьюдента

In [50]:
def student_dist(n, x):
    return sp.special.gamma((n+1)/2) * (np.pi*n)**(-0.5) * (sp.special.gamma(n/2))**(-1) * (1 + x**2 / n)**(-(n+1)/2)

def fisher_dist(d1, d2, x):
    return np.sqrt((d1*x / (d1*x + d2))**d1 * (d2 / (d1*x + d2))**d2) / (x*sp.special.gamma(d1/2)*sp.special.gamma(d2/2)/sp.special.gamma(d1/2+d2/2))

def normal_dist(a, std2, x):
    return 1 / np.sqrt(2*np.pi*std2) * np.exp( -(x - a)**2 / (2*std2) )

### Параметры задачи

In [51]:
# Таблица значений
data = {
    1: np.array([83, 85]),
    2: np.array([84, 85, 85, 86, 86, 87]),
    3: np.array([86, 87, 87, 87, 88, 88, 88, 88, 88, 89, 90]),
    4: np.array([89, 90, 90, 91]),
    5: np.array([90, 92])
}

# Количество факторов
p = k = 5

# Количество элементов выборки
n = np.sum([len(data[i]) for i in range(1, k+1)])

# Доверительное значение для принятия / отвержения гипотез
alpha = 0.05

### a) Определить влияние возраста на содержание иммуноглобина в крови с помощью регрессионного анализа.

Построим матрицу наблюдений $\Psi$ и вектор отклика $Y$.
В качестве факторов возьмем $\xi_k$, где $k=1..5$ - $\xi_k = 1$, если текущее значение иммуноглобина взято с возрастной группы $k$, иначе $\xi_k = 0$. 

In [52]:
Psi = np.zeros((n, k))
Y = np.zeros((n,))

i = 0
for k, v in data.items():
    for j in range(len(data[k])):
        Psi[i, k-1] = 1.0
        Y[i] = data[k][j]
        i += 1

Построим линейную регрессию и выведем уравнение регрессии.

In [53]:
lr = LinearRegression(Psi, Y)
beta_est = lr.beta_est()

print(f"𝜂 = ", end='')
for i in range(k):
    if i != 0:
        print(f" + ", end='')
    print(f"{round(beta_est[i], 4)}·𝜉{LOW_INDEX[i]}", end='')
print()

𝜂 = 84.0·𝜉₀ + 85.5·𝜉₁ + 87.8182·𝜉₂ + 90.0·𝜉₃ + 91.0·𝜉₄


Проверим значимости коэффициентов $\beta_k$.

In [54]:
F_inv = np.linalg.inv(lr.F())
RSS = lr.RSS()

for i in range(p):
    print(f"Исследование значимости коэффициента 𝛽{LOW_INDEX[i]}:")
    
    delta = beta_est[i] * np.sqrt(n - p) / np.sqrt(RSS * F_inv[i, i])
    print(f"Δ{LOW_INDEX[i]} = {round(delta, 4)}; ", end='')
    
    p_value = 2 * sp.integrate.quad(lambda x : student_dist(n - p, x), np.abs(delta), +math.inf)[0]
    print(f"p-value{LOW_INDEX[i]} = {round(p_value, 4)}", end=' ')
    
    if (p_value > alpha):
        print(f"=> Нет оснований отвергнуть гипотезу о незначимости коэффициента 𝛽{LOW_INDEX[i]}.")
    else:
        print(f"=> Отвергаем гипотезу о незначимости коэффициента 𝛽{LOW_INDEX[i]}.")
    print()

Исследование значимости коэффициента 𝛽₀:
Δ₀ = 110.449; p-value₀ = 0.0 => Отвергаем гипотезу о незначимости коэффициента 𝛽₀.

Исследование значимости коэффициента 𝛽₁:
Δ₁ = 194.7194; p-value₁ = 0.0 => Отвергаем гипотезу о незначимости коэффициента 𝛽₁.

Исследование значимости коэффициента 𝛽₂:
Δ₂ = 270.7997; p-value₂ = 0.0 => Отвергаем гипотезу о незначимости коэффициента 𝛽₂.

Исследование значимости коэффициента 𝛽₃:
Δ₃ = 167.3555; p-value₃ = 0.0 => Отвергаем гипотезу о незначимости коэффициента 𝛽₃.

Исследование значимости коэффициента 𝛽₄:
Δ₄ = 119.6531; p-value₄ = 0.0 => Отвергаем гипотезу о незначимости коэффициента 𝛽₄.



Определим коэффициенты $RSS$, $TSS$ и коэффициент детерминации регрессии $R^2$.

In [55]:
print(f"RSS = {round(lr.RSS(), 4)}")
print(f"TSS = {round(lr.TSS(), 4)}")
print(f"R² = {round(lr.R2(), 4)}")

RSS = 23.1364
TSS = 122.16
R² = 0.8106


Проверим значимость коэффициента детерминации $R^2$.

In [58]:
R2 = lr.R2()

delta = R2 / (1 - R2) * (n - p) / (p - 1)
print(f"Δ = {round(delta, 4)};", end=' ')

p_value = sp.integrate.quad(lambda x : fisher_dist(p - 1, n - p, x), delta, +math.inf)[0]
print(f"p-value = {p_value}", end=' ')

if p_value < alpha:
    print(f"=> Отвергаем гипотезу о незначимости коэффициента R².")
else:
    print(f"=> Нет оснований отвергнуть гипотезу о незначимости коэффициента R².")

Δ = 21.4; p-value = 5.407410946440232e-07 => Отвергаем гипотезу о незначимости коэффициента R².


### b) Провести попарное сравнение средних в рамках регрессионной модели.

In [68]:
F_inv = np.linalg.inv(lr.F())
RSS = lr.RSS()

for i in range(p-1):
    for j in range(i+1, p):
        print(f"Сравнение средних для возрастных групп {i+1} и {j+1}:")
        beta_i = beta_est[i]
        beta_j = beta_est[j]
        
        delta = (beta_i - beta_j) * np.sqrt(n - p) / np.sqrt(RSS * (F_inv[i, i] + F_inv[j, j]))
        print(f"Δ{LOW_INDEX[i+1]}{LOW_INDEX[j+1]} = {round(delta, 4)}")
        
        p_value = 2 * sp.integrate.quad(lambda x : student_dist(n - p, x), np.abs(delta), +math.inf)[0]
        print(f"p_value{LOW_INDEX[i+1]}{LOW_INDEX[j+1]} = {round(p_value, 4)}")
        
        if p_value < alpha:
            print(f"=> Отвергаем гипотезу однородности возрастных групп {i+1} и {j+1}.")
        else:
            print(f"=> Нет оснований отвергнуть гипотезу однородности возрастных групп {i+1} и {j+1}.")
        print()

Сравнение средних для возрастных групп 1 и 2:
Δ₁₂ = -1.7081
p_value₁₂ = 0.1031
=> Нет оснований отвергнуть гипотезу однородности возрастных групп 1 и 2.

Сравнение средних для возрастных групп 1 и 3:
Δ₁₃ = -4.6181
p_value₁₃ = 0.0002
=> Отвергаем гипотезу однородности возрастных групп 1 и 3.

Сравнение средних для возрастных групп 1 и 4:
Δ₁₄ = -6.4415
p_value₁₄ = 0.0
=> Отвергаем гипотезу однородности возрастных групп 1 и 4.

Сравнение средних для возрастных групп 1 и 5:
Δ₁₅ = -6.5083
p_value₁₅ = 0.0
=> Отвергаем гипотезу однородности возрастных групп 1 и 5.

Сравнение средних для возрастных групп 2 и 3:
Δ₂₃ = -4.2468
p_value₂₃ = 0.0004
=> Отвергаем гипотезу однородности возрастных групп 2 и 3.

Сравнение средних для возрастных групп 2 и 4:
Δ₂₄ = -6.4817
p_value₂₄ = 0.0
=> Отвергаем гипотезу однородности возрастных групп 2 и 4.

Сравнение средних для возрастных групп 2 и 5:
Δ₂₅ = -6.2629
p_value₂₅ = 0.0
=> Отвергаем гипотезу однородности возрастных групп 2 и 5.

Сравнение средних для во